In [ ]:
import spotipy
import urllib.request
import random
import requests
import nltk
import webbrowser
import json
from nltk import pos_tag
import spotipy.util as util

In [ ]:
category = "technology"
country = "gb"

In [ ]:
with open("news_key.txt", "r") as file:
    key = file.read()

In [ ]:
url = "https://newsapi.org/v2/top-headlines?country=" + country + "&category=" + category + "&apiKey=" + key

In [ ]:
url

In [ ]:
# requesting the information
request = urllib.request.Request(url)
# the information is coming back to me
response = urllib.request.urlopen(request)

In [ ]:
response

In [ ]:
# to get the data out of the response object
headlines = json.loads(response.read())

In [ ]:
headlines

In [ ]:
headlines['articles'][0]['title']

In [ ]:
article_titles = [article['title'] for article in headlines['articles']]

In [ ]:
article_titles

In [ ]:
tagged = pos_tag(article_titles[0].split())
tagged

In [ ]:
for tag in tagged:
    print(tag)

In [ ]:
important_words = []
for article in article_titles:
    tagged = pos_tag(article.split())
    # only select the proper nouns
    important_words.extend([word for word,tag in tagged if tag == 'NNP'])

important_words

In [ ]:
with open("spotify_keys.json", "r") as file:
    api_tokens = json.load(file)

In [ ]:
client_id = api_tokens["client_id"]
client_secret = api_tokens["client_secret"]
redirectURI = api_tokens["redirect"]
username = api_tokens["username"]

In [ ]:
scope = 'user-read-private user-read-playback-state user-modify-playback-state playlist-modify-public user-library-read'
token = util.prompt_for_user_token(username, scope, client_id=client_id,
                           client_secret=client_secret,
                           redirect_uri=redirectURI)

In [ ]:
token

In [ ]:
sp = spotipy.Spotify(auth=token)

In [ ]:
# using the artist filter
searchResults = sp.search(q="artist:" + important_words[0], type="track", limit=1)

In [ ]:
important_words[0]

In [ ]:
searchResults['tracks']['items']

In [ ]:
tracks = []
for word in important_words:
    searchResults = sp.search(q="artist:" + word, type="track", limit=1)
    # does the search return any results?
    if len(searchResults['tracks']['items']) > 0:
        tracks.append(searchResults['tracks']['items'][0]['uri'])

tracks

In [ ]:
# creating a new playlist - but has no songs!
my_playlist = sp.user_playlist_create(user=username, name="Todays News in " + category +  "in" + country, public=True,
                                      description="Songs for the news")


In [ ]:
# we are adding songs to our playlist!
results = sp.user_playlist_add_tracks(username, my_playlist['id'], tracks)

In [ ]:
my_playlist

In [ ]:
webbrowser.open(my_playlist['external_urls']['spotify'])